In [3]:
def find_antinodes(grid):
    # Find all antennas
    antennas = {}
    for y, row in enumerate(grid):
        for x, char in enumerate(row):
            if char != '.' and char != '#':
                # Group antennas by their frequency
                if char not in antennas:
                    antennas[char] = []
                antennas[char].append((x, y))
    
    # Set to store unique antinode locations
    antinodes = set()
    
    # Check antinodes for each frequency
    for freq, antenna_list in antennas.items():
        # Check all pairs of antennas with the same frequency
        for i in range(len(antenna_list)):
            for j in range(i+1, len(antenna_list)):
                x1, y1 = antenna_list[i]
                x2, y2 = antenna_list[j]
                
                # Calculate midpoint
                mid_x = (x1 + x2) / 2
                mid_y = (y1 + y2) / 2
                
                # Calculate antinode locations
                dx = x2 - x1
                dy = y2 - y1
                
                # First antinode (on one side)
                antinode1_x = int(mid_x + dy)
                antinode1_y = int(mid_y - dx)
                
                # Second antinode (on other side)
                antinode2_x = int(mid_x - dy)
                antinode2_y = int(mid_y + dx)
                
                # Only add antinodes within grid bounds
                if 0 <= antinode1_x < len(grid[0]) and 0 <= antinode1_y < len(grid):
                    antinodes.add((antinode1_x, antinode1_y))
                
                if 0 <= antinode2_x < len(grid[0]) and 0 <= antinode2_y < len(grid):
                    antinodes.add((antinode2_x, antinode2_y))
    
    return len(antinodes)

def solve_antenna_problem(filename):
    # Read the input file
    with open(filename, 'r') as file:
        grid = [line.strip() for line in file]
    
    # Calculate and return the number of unique antinode locations
    return find_antinodes(grid)

# Example usage
def main():
    # Prompt user for input file
    filename = input("Enter the path to the input file: ")
    
    try:
        # Solve the problem
        result = solve_antenna_problem(filename)
        print(f"Number of unique antinode locations: {result}")
    
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the main function
if __name__ == "__main__":
    main()

Number of unique antinode locations: 464


In [5]:
import re

def read_map_from_file(filename):
    """Reads the map from the input file."""
    with open(filename, 'r') as file:
        return [line.strip() for line in file.readlines()]

def find_antennas(map_data):
    """Finds all antennas in the map and their positions."""
    antennas = {}
    for y, row in enumerate(map_data):
        for x, char in enumerate(row):
            if re.match(r'[a-zA-Z0-9]', char):
                if char not in antennas:
                    antennas[char] = []
                antennas[char].append((x, y))
    return antennas

def calculate_antinodes(antennas, map_width, map_height):
    """Calculates unique antinode positions within the map bounds."""
    antinodes = set()

    for frequency, positions in antennas.items():
        n = len(positions)
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                x1, y1 = positions[i]
                x2, y2 = positions[j]

                # Calculate differences
                dx, dy = x2 - x1, y2 - y1

                # Check if midpoint forms valid antinodes
                mid_x1, mid_y1 = x1 - dx, y1 - dy
                mid_x2, mid_y2 = x2 + dx, y2 + dy

                # Add both potential antinodes if they are in bounds
                if 0 <= mid_x1 < map_width and 0 <= mid_y1 < map_height:
                    antinodes.add((mid_x1, mid_y1))
                if 0 <= mid_x2 < map_width and 0 <= mid_y2 < map_height:
                    antinodes.add((mid_x2, mid_y2))

    return antinodes

def main():
    filename = "puzzle_input.txt"  # Replace with your file name
    map_data = read_map_from_file(filename)
    
    # Map dimensions
    map_width = len(map_data[0])
    map_height = len(map_data)

    # Find antennas and calculate antinodes
    antennas = find_antennas(map_data)
    antinodes = calculate_antinodes(antennas, map_width, map_height)

    # Count unique antinode locations
    print("Total unique antinode locations:", len(antinodes))

if __name__ == "__main__":
    main()

Total unique antinode locations: 396


In [10]:
import re
from collections import defaultdict

def read_map_from_file(filename):
    """Reads the map from the input file."""
    with open(filename, 'r') as file:
        return [line.strip() for line in file.readlines()]

def find_antennas(map_data):
    """Finds all antennas in the map and their positions."""
    antennas = defaultdict(list)
    for y, row in enumerate(map_data):
        for x, char in enumerate(row):
            if re.match(r'[a-zA-Z0-9]', char):
                antennas[char].append((x, y))
    return antennas

def get_line_points(p1, p2, map_width, map_height):
    """Get all points on a line within map bounds."""
    x1, y1 = p1
    x2, y2 = p2
    points = set()
    
    # Calculate direction vector
    dx = x2 - x1
    dy = y2 - y1
    
    # Normalize to get unit vector components
    # Handle vertical, horizontal, and diagonal lines
    if dx == 0:  # Vertical line
        step_x, step_y = 0, 1 if dy > 0 else -1
    elif dy == 0:  # Horizontal line
        step_x, step_y = 1 if dx > 0 else -1, 0
    else:
        # Check if diagonal (must be 45 degrees)
        if abs(dx) != abs(dy):
            return points
        step_x = 1 if dx > 0 else -1
        step_y = 1 if dy > 0 else -1
    
    # Start at map bounds and traverse to opposite bounds
    x, y = 0, 0
    if step_x == 0:  # Vertical line
        x = x1
        for y in range(map_height):
            points.add((x, y))
    elif step_y == 0:  # Horizontal line
        y = y1
        for x in range(map_width):
            points.add((x, y))
    else:  # Diagonal line
        # Find starting point at map boundary
        if abs(step_x) == 1 and abs(step_y) == 1:
            # Calculate starting position
            if step_x == step_y:  # Main diagonal
                y_start = 0
                x_start = x1 - y1
                while x_start < 0:
                    x_start += 1
                    y_start += 1
            else:  # Anti-diagonal
                y_start = 0
                x_start = x1 + y1
                while x_start >= map_width:
                    x_start -= 1
                    y_start += 1
            
            x, y = x_start, y_start
            while 0 <= x < map_width and 0 <= y < map_height:
                points.add((x, y))
                x += step_x
                y += step_y
    
    return points

def calculate_antinodes(antennas, map_width, map_height):
    """Calculates unique antinode positions based on resonant harmonics."""
    antinodes = set()
    
    # Process each frequency group
    for freq, positions in antennas.items():
        # Skip frequencies with only one antenna
        if len(positions) < 2:
            continue
        
        # Add all antenna positions as antinodes
        antinodes.update(positions)
        
        # Check all pairs of antennas
        for i, p1 in enumerate(positions):
            for j, p2 in enumerate(positions[i+1:], i+1):
                # Get all points on the line between these antennas
                line_points = get_line_points(p1, p2, map_width, map_height)
                antinodes.update(line_points)
    
    return antinodes

def main():
    filename = "puzzle_input.txt"
    map_data = read_map_from_file(filename)
    
    map_width = len(map_data[0])
    map_height = len(map_data)
    
    antennas = find_antennas(map_data)
    antinodes = calculate_antinodes(antennas, map_width, map_height)
    
    print("Total unique antinode locations:", len(antinodes))

if __name__ == "__main__":
    main()

Total unique antinode locations: 326
